# Workshop 12.3 GPT

In [ ]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 6.7 MB/s 
     |████████████████████████████████| 5.8 MB 47.9 MB/s 
     |████████████████████████████████| 132 kB 51.4 MB/s 
     |████████████████████████████████| 212 kB 60.8 MB/s 
     |████████████████████████████████| 182 kB 64.1 MB/s 
     |████████████████████████████████| 127 kB 75.1 MB/s 
     |████████████████████████████████| 7.6 MB 43.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import torch
import torch.nn.functional as F

Загрузим модель для русского языка из Huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokens = tokenizer("Вопрос: 'Сколько будет 2+2?'\nОтвет:")
tokens

{'input_ids': [53, 30, 985, 784, 16794, 6656, 282, 24501, 4666, 282, 644, 40360, 461, 35, 776, 30], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
with torch.no_grad():
    result = model(input_ids = tokens.input_ids)
result.keys()

odict_keys(['logits', 'past_key_values'])

# Beam search

Самый простой способ генерации текста — это greedy search. Чуть более сложный и качественный способ сэмплирования — это beam search. В этом случае на каждом шаге мы выбираем не только один самый вероятный токен, а сразу несколько (beam-size), и дальше продолжаем поиск для каждого из выбранных токенов.

In [ ]:
beam = [(0.0, tokens.input_ids)]

In [ ]:
batch = torch.tensor([c[1] for c in beam])

In [ ]:
with torch.no_grad():
    result = model(input_ids = tokens.input_ids)
result.keys()

In [ ]:
probs_batch = result.logits[:, -1].log_softmax(1)
probs_batch

tensor([[-22.8702, -20.4897, -21.7028,  ..., -22.9258, -22.5695, -22.9189]])

In [ ]:
def find_top(beam, probs_batch, num_beams):
    results = []

    for i, p in enumerate(probs_batch.numpy()):
        base_prob, base_tokens = beam[i]
        for token_index, token_prob in enumerate(p): 
            results.append((base_prob + token_prob, (base_tokens + [token_index])))

    results.sort()

    return results[-(num_beams):]

In [ ]:
num_beams = 8

In [ ]:
for r in find_top(beam, probs_batch, num_beams):
    print(tokenizer.decode(r[1]))

Q: Как мне вывести список в обратном порядке в python? A: Попробуйте
Q: Как мне вывести список в обратном порядке в python? A: Можно
Q: Как мне вывести список в обратном порядке в python? A: Есть
Q: Как мне вывести список в обратном порядке в python? A: Как
Q: Как мне вывести список в обратном порядке в python? A: Если
Q: Как мне вывести список в обратном порядке в python? A: У
Q: Как мне вывести список в обратном порядке в python? A: Для
Q: Как мне вывести список в обратном порядке в python? A: В


In [ ]:
def beam_step(beam, num_beams):
    batch = torch.tensor([c[1] for c in beam])
    with torch.no_grad():
        result = model(input_ids = batch)

    log_probs_batch = result.logits[:, -1].log_softmax(1)

    return find_top(beam, log_probs_batch, num_beams)

In [ ]:
beam = beam_step(beam, num_beams)

for r in beam:
    print(tokenizer.decode(r[1]))

Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующие действия:/public static class
Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующее:/public static final int
Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующие действия:/public static final
Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующие действия:/public static v
Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующее:/public static void
Q: Как мне вывести список в обратном порядке в python? A: Для того, чтобы вывести список в обратном порядке в python, нужно выполнить следующие дейст

In [ ]:
def run_beam(text, num_beams, max_length=128):
    tokens = tokenizer(text)
    beam = [(0.0, tokens.input_ids)]

    for i in tqdm(range(max_length)):
        beam = beam_step(beam, num_beams)

    return tokenizer.decode(beam[-1][1]))

SyntaxError: ignored

In [ ]:
inputs = tokenizer("Мороз и солнце; день чудесный ", return_tensors="pt")

outputs = model.generate(
    **inputs,
    num_beams=5,
    no_repeat_ngram_size=4,
    max_length=128,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
model()

tensor([[ 1464, 10627, 30952,   289, 41867,  3327,    18,   203,   203,   384,
           369,  4529,    16,   369,   384,  1109, 41867,  3327,    16,   369,
           384,   515,   481,   322,  2862,    16,   374,   656,   322,  5087,
           953, 11680,   282,   970,    16,   374,   745,   322, 10906, 15894,
           282,   813,    18,   203,   203,   203,    14,  2925,  2925,   203,
           203,   677,  1500,  1119,    16,   785, 41867,  3327,   289, 41867,
          3327,  9811,   282, 11226,    17, 35649,   344,    16,   745,   898,
           726,   309,  2468,    18,   203,   203,  2176, 38974, 12183,  6118,
            17, 33074,  1034, 10627, 30952,  5739,   282,  3177, 12719,    18,
           203,   203,  1464, 10627, 30952,  6544,   294, 41867,  3327,   272,
           289,  1109,    30,   203,   203,   384,  1012,  5428,    16,   374,
           417,   322,  2447,   953,  3893,    16,   374,   387,   322, 37517,
         15894,   282,   703,    18,   203,   203,  

In [ ]:
print(f"Output: {tokenizer.batch_decode(outputs)[0]}")

Output: Штирлиц и Мюллер.

— Да, — сказал Мюллер, — но это не значит, что мы не можем быть уверены в том, что они не станут стрелять в нас.


* * *

В тот день, когда Мюллер и Мюллер прибыли в штаб-квартиру, они были уже на месте.

Начальник штаба генерал-лейтенант Штирлиц сидел в своем кабинете.

Штирлиц подошел к Мюллеру и сказал:

— Вы знаете, что я не могу быть уверен, что вы не станете стрелять в меня.

Мюл
